In [9]:
import os
import glob
import pandas as pd
import h5py

In [10]:
csvfile = pd.read_csv('exptSheet.csv')

In [11]:
csvfile.head()

,Experiment date,Setup ID,Experiment Name,Male flye type,Male DOB,Male days old,Female fly type,Female DOB,Female days old,Temperature,Humidity,Comments,Proofread Status,Notes,Exclude(?)
0,2023-05-12,Left,20230512_153214,NM91,2023-05-08,4,lexAOP-Flp/UAS>stop>TNT;R42B01-Gal4/dsx-lexA,2023-05-05,7,23.0,40%,Camera frozen,skipped,NaN,NaN
1,2023-05-12,Right,20230512_153214,NM91,2023-05-08,4,lexAOP-Flp/+;R42B01-Gal4/dsx-lexA,2023-05-05,7,23.0,40%,"sigma coat is not painted well, discard this",skipped,NaN,NaN
2,2023-05-12,Left,20230512_163844,NM91,2023-05-08,4,lexAOP-Flp/UAS>stop>TNT;R42B01-Gal4/dsx-lexA,2023-05-05,7,23.0,40%,Camera frozen,skipped,NaN,NaN
3,2023-05-12,Right,20230512_163844,NM91,2023-05-08,4,lexAOP-Flp/+;R42B01-Gal4/dsx-lexA,2023-05-05,7,23.0,40%,"sigma coat is not painted well, discard this",skipped,NaN,NaN
4,2023-05-15,Left,20230515_111851,NM91,2023-05-10,5,lexAOP-Flp/UAS>stop>TNT;R42B01-Gal4/dsx-lexA,2023-05-10,5,23.5,37%,wing is damaged,skipped,NaN,NaN


In [12]:
csvfile['Female fly type'].unique()

array(['lexAOP-Flp/UAS>stop>TNT;R42B01-Gal4/dsx-lexA',
       'lexAOP-Flp/+;R42B01-Gal4/dsx-lexA'], dtype=object)

In [13]:
dataDir = '/run/user/1000/gvfs/smb-share:server=cup.pni.princeton.edu,share=murthy/Kyle/flies/pc2l_tnt/'

In [14]:
metadata = {}
for idx in range(len(csvfile)):

    row = csvfile.loc[idx]
    name = row['Experiment Name']
    rig = row['Setup ID']

    exptname = name+rig.lower()
    exptDate = row['Experiment date']
    maleDOB = row['Male DOB']
    femaleDOB = row['Female DOB']

    exptpath = os.path.join(dataDir, name, rig.lower(),exptname+'.h5')
    if not os.path.exists(exptpath):
        print(exptpath)
        continue
    else:
        with h5py.File(exptpath, 'r') as f:
            mfDist = f['mfDist'][:]
    copulated = len(mfDist) < 108000

    cop = int(copulated)
    if copulated:
        copFrame = len(mfDist)
    else:
        copFrame = 'NaN'

    if row['Female fly type'] == 'lexAOP-Flp/UAS>stop>TNT;R42B01-Gal4/dsx-lexA':
        femaleGT = 'pc2_tnt'
    else:
        femaleGT = 'pc2_control'

    maleGT = row['Male flye type']

    maleAge = row['Male days old']
    femaleAge = row['Female days old']

#     metaDict = {'exptName':expt, 'exptDate':exptDate,
#                 'maleGT':maleGT, 'maleDOB':maleDOB, 'maleAge':maleAge,
#                 'femaleGT':femaleGT,'femaleDOB':femaleDOB, 'femaleAge':femaleAge,
#                 'cop':cop, 'copFrame':copFrame}
#     metadata |= metaDict
    metadata[idx] = {'exptName':exptname, 'exptDate':exptDate,
                      'maleGT':maleGT, 'maleDOB':maleDOB, 'maleAge':maleAge,
                      'femaleGT':femaleGT,'femaleDOB':femaleDOB, 'femaleAge':femaleAge,
                      'cop':cop, 'copFrame':copFrame}

/run/user/1000/gvfs/smb-share:server=cup.pni.princeton.edu,share=murthy/Kyle/flies/pc2l_tnt/20230512_153214/left/20230512_153214left.h5
/run/user/1000/gvfs/smb-share:server=cup.pni.princeton.edu,share=murthy/Kyle/flies/pc2l_tnt/20230512_153214/right/20230512_153214right.h5
/run/user/1000/gvfs/smb-share:server=cup.pni.princeton.edu,share=murthy/Kyle/flies/pc2l_tnt/20230512_163844/left/20230512_163844left.h5
/run/user/1000/gvfs/smb-share:server=cup.pni.princeton.edu,share=murthy/Kyle/flies/pc2l_tnt/20230512_163844/right/20230512_163844right.h5
/run/user/1000/gvfs/smb-share:server=cup.pni.princeton.edu,share=murthy/Kyle/flies/pc2l_tnt/20230515_111851/left/20230515_111851left.h5
/run/user/1000/gvfs/smb-share:server=cup.pni.princeton.edu,share=murthy/Kyle/flies/pc2l_tnt/20230515_111851/right/20230515_111851right.h5
/run/user/1000/gvfs/smb-share:server=cup.pni.princeton.edu,share=murthy/Kyle/flies/pc2l_tnt/20230517_094418/right/20230517_094418right.h5


In [15]:
meta = pd.DataFrame.from_dict(metadata, orient='index')

In [16]:
meta.to_csv('metadata.csv')